In [1]:
import os
import subprocess

def convert_shapefile_to_pmtiles(shapefile_path, pmtiles_path, min_zoom=10, max_zoom=15):
    """
    Converts a shapefile to PMTiles format using GDAL's ogr2ogr.
    
    Parameters:
        shapefile_path (str): Path to the shapefile (.shp).
        pmtiles_path (str): Path to save the PMTiles output.
        min_zoom (int): Minimum zoom level for PMTiles.
        max_zoom (int): Maximum zoom level for PMTiles.
    """
    try:
        # Run the ogr2ogr command to convert to PMTiles
        subprocess.run([
            "ogr2ogr",
            "-dsco", f"MINZOOM={min_zoom}",
            "-dsco", f"MAXZOOM={max_zoom}",
            "-f", "PMTiles",
            pmtiles_path,
            shapefile_path
        ], check=True)
        print(f"PMTiles created at {pmtiles_path}")
    except subprocess.CalledProcessError as e:
        print(f"Error processing {shapefile_path}: {e}")

def create_pmtiles_for_directory(shapefile_dir, pmtiles_dir, min_zoom=10, max_zoom=10):
    """
    Scans a directory of shapefiles and generates PMTiles for each.
    
    Parameters:
        shapefile_dir (str): Directory containing shapefiles (including subdirectories).
        pmtiles_dir (str): Directory to save the generated PMTiles files.
        min_zoom (int): Minimum zoom level for PMTiles.
        max_zoom (int): Maximum zoom level for PMTiles.
    """
    # Ensure the PMTiles output directory exists
    os.makedirs(pmtiles_dir, exist_ok=True)
    
    # Gather all shapefiles recursively
    shapefiles = []
    for root, dirs, files in os.walk(shapefile_dir):
        for file in files:
            if file.endswith(".shp"):
                shapefiles.append(os.path.join(root, file))
    
    print(f"Found {len(shapefiles)} shapefiles to process.")

    # Process each shapefile
    for shapefile_path in shapefiles:
        # Define the output PMTiles path based on the shapefile name
        pmtiles_name = f"{os.path.splitext(os.path.basename(shapefile_path))[0]}.pmtiles"
        pmtiles_path = os.path.join(pmtiles_dir, pmtiles_name)
        
        # Convert shapefile to PMTiles
        convert_shapefile_to_pmtiles(shapefile_path, pmtiles_path, min_zoom, max_zoom)

# Usage
shapefile_directory = "Land use_2002"
pmtiles_directory = "Land use_2002_pmtiles"

create_pmtiles_for_directory(shapefile_directory, pmtiles_directory)


Found 24 shapefiles to process.
PMTiles created at Land use_2002_pmtiles/cecilu02.pmtiles
PMTiles created at Land use_2002_pmtiles/montlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/calvlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/garrlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/prinlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/carrlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/dorclu02.pmtiles
PMTiles created at Land use_2002_pmtiles/annelu02.pmtiles
PMTiles created at Land use_2002_pmtiles/queelu02.pmtiles
PMTiles created at Land use_2002_pmtiles/harflu02.pmtiles
PMTiles created at Land use_2002_pmtiles/carolu02.pmtiles
PMTiles created at Land use_2002_pmtiles/bacolu02.pmtiles
PMTiles created at Land use_2002_pmtiles/charlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/kentlu02.pmtiles
PMTiles created at Land use_2002_pmtiles/somelu02.pmtiles
PMTiles created at Land use_2002_pmtiles/worclu02.pmtiles
PMTiles created at Land use_2002_pmtiles